# Cuaderno de Documentación: 03 - Gestión de la Base de Conocimiento Vectorial (`vector_db_manager.py`)

El módulo `scripts/vector_db_manager.py` es responsable de la creación, carga y gestión de la base de datos vectorial, que es un componente esencial del sistema RAG. Esta base de datos almacena los "embeddings" (representaciones vectoriales numéricas) de los fragmentos de texto de nuestra base de conocimiento, permitiendo búsquedas eficientes por similitud semántica.

## Funcionalidades Principales

1.  **`get_embedding_function(model_name)`:**
    * **Propósito:** Inicializar y devolver la función de embeddings que se utilizará para convertir texto en vectores.
    * **Proceso:**
        * Utiliza `SentenceTransformerEmbeddings` de LangChain, que a su vez carga un modelo pre-entrenado de la biblioteca `sentence-transformers` (por ejemplo, `all-MiniLM-L6-v2` especificado en `config.py`).
        * Configura el modelo para que se ejecute en la CPU (`model_kwargs={'device': 'cpu'}`) por defecto, lo que asegura la compatibilidad en entornos sin GPU dedicada. Si se dispone de una GPU compatible con CUDA, esto podría cambiarse a `'cuda'` para acelerar la generación de embeddings (aunque para `all-MiniLM-L6-v2` la diferencia puede no ser drástica en bases de conocimiento pequeñas).
    * **Retorno:** Una instancia de la función de embeddings, o `None` si ocurre un error durante la inicialización.

2.  **`crear_o_cargar_chroma_db(chroma_db_path, docs_base_conocimiento_path, embedding_function, chunk_size, chunk_overlap, recrear_db_flag)`:**
    * **Propósito:** Gestionar la base de datos vectorial ChromaDB. Puede crear una nueva base de datos a partir de los documentos de la base de conocimiento o cargar una existente desde el disco.
    * **Proceso:**
        * **Recreación (si `recrear_db_flag` es `True` o la DB no existe):**
            * Si `recrear_db_flag` es `True` y la carpeta `chroma_db_path` existe, se elimina su contenido para asegurar una creación limpia.
            * Llama a `document_utils.cargar_y_procesar_pdfs_de_carpeta()` para obtener los fragmentos de texto de los documentos en `docs_base_conocimiento_path`.
            * Si se obtienen fragmentos, utiliza `Chroma.from_documents()` para:
                * Generar embeddings para cada fragmento usando la `embedding_function` proporcionada.
                * Almacenar estos fragmentos y sus embeddings en una nueva colección de ChromaDB.
                * Persistir (guardar) la base de datos en el directorio especificado por `chroma_db_path`. Las versiones recientes de ChromaDB (>0.4.0) manejan la persistencia automáticamente al crear la instancia con `persist_directory`.
        * **Carga (si la DB existe y `recrear_db_flag` es `False`):**
            * Utiliza `Chroma()` para cargar la base de datos persistida desde `chroma_db_path`, especificando la misma `embedding_function` que se usó para crearla.
            * Realiza una verificación básica para asegurar que la base de datos cargada contiene elementos.
    * **Retorno:** Una instancia de la base de datos vectorial Chroma (`Chroma`), o `None` si ocurre un error.

## Importancia de la Persistencia

ChromaDB guarda la base de datos vectorial en el disco (en la carpeta especificada por `CHROMA_DB_PATH`, por defecto `datos/ChromaDB_V1/`). Esto significa que el costoso proceso de cargar PDFs, fragmentarlos y generar embeddings solo necesita realizarse una vez (o cuando los documentos de la base de conocimiento cambien). En ejecuciones posteriores, el sistema puede cargar rápidamente la base de datos preexistente.

A continuación, se muestra el contenido del script `vector_db_manager.py` para referencia.

In [ ]:
# Cargar el contenido del script vector_db_manager.py
%load ../scripts/vector_db_manager.py